Firstly we will import all the dependencies 

In [1]:
import json
import os
# import shutil
import cv2
import numpy as np
import os
# from matplotlib import pyplot as plt
# import time
# import mediapipe as mp
# import re

In [2]:
# video_data_folder = "../VideoData"
bds_folder = "../BdSLW60"
numerical_data_path='../NumericalData'

In [10]:
# Step 2: Open the JSON file
datas = []
word_lists = os.listdir(bds_folder)
for word in word_lists:
    if word.endswith('.pdf') or word.endswith('.xlsx') or word.endswith('.txt'):
        pass 
    else:
        with open(f'{bds_folder}/{word}/output1.json', 'r') as file:
            # Step 3: Parse the JSON data into a Python data structure
            datas.append(json.load(file))
# for data in datas:
#     try:
#         print(data['U11'])
#     except:
#         pass

In [17]:
import math

MAX_FRAME=164
testUser = {'U4','U8'}
relativePath='RELATIVE'

In [18]:
def makeRelativeAndFlatten(npy_mat):
    
    #if a point is missing do no calibration or relative translation on it

    #hand points will be translated relative to wrist point
    #wrist point will be ralative to elbow point
    #elbow point will be relative to shoulder point
    

    #shoulder points will be relatvie to their middle positions, this is already done during calibration with respect to first frame of the first sign of the video

    #face points will be relative to nose point
    #nose point will be relative to the middle of two shoulder points
    
    #@will not touch these foot parts as they are zeros in our case
    #heel and foot index point will be relative to ankle point
    #ankle point will be ralative to knee point
    #knee point will be relative to heap points 
    # heap points will be relative to the middle of two shoulder points, already done during calibratoin
    
 

    point=542
    for i in range (0,543):
        x=npy_mat[point][0]
        y=npy_mat[point][1]
        d=npy_mat[point][2]
        
        if(x<=0.0 and y <=0.0 and d<=0.0) :
            point=point-1
            continue

        ref_x= 0
        ref_y= 0
        ref_d= 0
        if point in range (522,543): #right hand points
            ref_x= npy_mat[16][0] #ref right wrist
            ref_y= npy_mat[16][1]
            ref_d= npy_mat[16][2]
        elif point in range (501,522): #left hand
            ref_x= npy_mat[15][0] #ref left wrist
            ref_y= npy_mat[15][1]
            ref_d= npy_mat[15][2]
        elif point in range (33,501): #face points
            ref_x= npy_mat[0][0] # ref nose
            ref_y= npy_mat[0][1]
            ref_d= npy_mat[0][2]
        elif point in range (0,33):
            #point value comes from 32 to 0 as loop designed
            if point ==0: # nose   ref is middle of two shoulder
                ref_x= (npy_mat[11][0]+npy_mat[12][0])/2
                ref_y= (npy_mat[11][1]+npy_mat[12][1])/2
                ref_d= (npy_mat[11][2]+npy_mat[12][2])/2
            #as loop comes from 32 to 0, face points will come fist before nose value is translated            
            elif point in range (1,11): # face points ref is nose point
                ref_x= npy_mat[0][0] 
                ref_y= npy_mat[0][1]
                ref_d= npy_mat[0][2]
            elif point in range (11,13):  # two shoulder points do nothing as ref is initialized to zero first
                ref_x= 0
                ref_y= 0
                ref_d= 0
            #elbow and wrist comes before shoulder points as we loop 542 to 0
            elif point ==13: #lefthand elbow 
                #ref is left shoulder point 
                ref_x= npy_mat[11][0]
                ref_y= npy_mat[11][1]
                ref_d= npy_mat[11][2]

            elif point ==15: #lefthand wrist
                #ref is left elbow 
                ref_x= npy_mat[13][0]
                ref_y= npy_mat[13][1]
                ref_d= npy_mat[13][2]
            elif point ==14: #righht elbow 
                #ref is righht shoulder point 
                ref_x= npy_mat[12][0]
                ref_y= npy_mat[12][1]
                ref_d= npy_mat[12][2]

            elif point ==16: #righht wrist
                #ref is righht elbow 
                ref_x= npy_mat[14][0]
                ref_y= npy_mat[14][1]
                ref_d= npy_mat[14][2]
            # finger tips come before wrist as we loop from high to low
            elif point in range (17,23) and point %2 ==1: #lefthand  finger tips
                #ref is left wrist
                ref_x= npy_mat[15][0]
                ref_y= npy_mat[15][1]
                ref_d= npy_mat[15][2]
            elif point in range (17,23) and point %2 ==0: #righthand  finger tips
                #ref is right wrist
                ref_x= npy_mat[16][0]
                ref_y= npy_mat[16][1]
                ref_d= npy_mat[16][2]
            elif point in range (23,25):  # two hips do nothing
                ref_x= 0
                ref_y= 0
                ref_d= 0
            elif point ==25: #left kneeee 
                #ref is left hip 
                ref_x= npy_mat[23][0]
                ref_y= npy_mat[23][1]
                ref_d= npy_mat[23][2]

            elif point ==27: #left ankle
                #ref is left kneee
                ref_x= npy_mat[25][0]
                ref_y= npy_mat[25][1]
                ref_d= npy_mat[25][2]
            elif point ==29: #left heel
                #ref is left anklee
                ref_x= npy_mat[27][0]
                ref_y= npy_mat[27][1]
                ref_d= npy_mat[27][2]

            elif point ==31: #left index
                #ref is left anklee
                ref_x= npy_mat[27][0]
                ref_y= npy_mat[27][1]
                ref_d= npy_mat[27][2]


            elif point ==26: #rightt kneeee 
                #ref is righhtt hip 
                ref_x= npy_mat[24][0]
                ref_y= npy_mat[24][1]
                ref_d= npy_mat[24][2]

            elif point ==28: #righhtt ankle
                #ref is rightt kneee
                ref_x= npy_mat[26][0]
                ref_y= npy_mat[26][1]
                ref_d= npy_mat[26][2]

            elif point ==30: #righhtt heel
                #ref is rightt hip
                ref_x= npy_mat[28][0]
                ref_y= npy_mat[28][1]
                ref_d= npy_mat[28][2]

            elif point ==32: #righhtt index
                #ref is rightt hip
                ref_x= npy_mat[28][0]
                ref_y= npy_mat[28][1]
                ref_d= npy_mat[28][2]
            
        npy_mat[point][0] =x-ref_x
        npy_mat[point][1] =y-ref_y
        npy_mat[point][2] = d-ref_d
        point=point-1




    return npy_mat.flatten()

In [19]:

for data in datas:
    for word in data:
        for user in data[word]:
            for orientation in data[word][user]:
                 for fileName in data[word][user][orientation]:
                    
                    frameRate = data[word][user][orientation][fileName]['FrameRate']
                    no_trials = len(data[word][user][orientation][fileName]['trials'])
                    
                    #override the trial count as sometime that is missing in annotation
                    data[word][user][orientation][fileName]['no_of_trials']= no_trials

                    #fileName_text = data[word][user][orientation][fileName]['FileName']
                    #orientation_text = data[word][user][orientation][fileName]['Orientation']
                    
                    data_path = f'{numerical_data_path}'
                    destination_path = f'{data_path}/{relativePath}'
                    if user in  testUser:
                        data_path = data_path+'/Test'
                        destination_path = destination_path+'/Test'
                    else:
                        data_path = data_path+'/Training'
                        destination_path = destination_path+'/Training'

                    lhFlag = False
                    if(orientation == 'LeftHand'):
                        lhFlag = True
                        flip_destination_path = f'{destination_path}/{orientation}_FLIPPED/{word}/{user}/{fileName}' 
                        flip_cal_destination_path = f'{destination_path}/{orientation}_FLIPPED_CALIB/{word}/{user}/{fileName}' 
                        #print(flip_destination_path)
                    #print(flip_destination_path)
                    #print(lhFlag)
                    data_path = f'{data_path}/{orientation}/{word}/{user}/{fileName}'
                    destination_path = f'{destination_path}/{orientation}/{word}/{user}/{fileName}'                          
                    
                    
                    

                    ref_x=0
                    ref_y=0
                    ref_d=0
                    i=0
                    for i in range(0,no_trials):
                        src_trial_folder = f'{data_path}/t{i}'
                        dest_trial_folder = f'{destination_path}/t{i}'
                        if os.path.exists(dest_trial_folder):
                            pass
                        else:
                            os.makedirs(dest_trial_folder)
                        #print(lhFlag)
                        if(lhFlag == True):
                            #print('left hand trial folder creation if')
                            flip_dest_trial_folder = f'{flip_destination_path}/t{i}'
                            flip_cal_dest_trial_folder = f'{flip_cal_destination_path}/t{i}'
                            # if os.path.exists(flip_dest_trial_folder):
                            #     #print(flip_dest_trial_folder,' exists')
                            #     pass
                            # else:
                            #     os.makedirs(flip_dest_trial_folder)
                            #     #print(flip_dest_trial_folder, 'created')

                            if os.path.exists(flip_cal_dest_trial_folder):
                                #print(flip_dest_trial_folder,' exists')
                                pass
                            else:
                                os.makedirs(flip_cal_dest_trial_folder)    
                            
                        starting = data[word][user][orientation][fileName]['trials'][str(i)]['starting']
                        ending = data[word][user][orientation][fileName]['trials'][str(i)]['ending']
                        # if first trial, starting frame will be calibration frame
                        
                        if i == 0:
                            #load the starting npy
                            starting_npy=f'{src_trial_folder}/{starting}.npy'
                            #print(starting_npy)
                            calib_npy=np.load(starting_npy);
                            #print(calib_npy)
                            #pose 
                            #face 
                            #lh 
                            #rh 
                            keypoints3D=calib_npy.reshape(-1,3)
                            left_shoulder = keypoints3D[11]
                            right_shoulder = keypoints3D[12]
                            ref_x=(left_shoulder[0]+right_shoulder[0])/2
                            ref_y=(left_shoulder[1]+right_shoulder[1])/2
                            ref_d=(left_shoulder[2]+right_shoulder[2])/2
                            #print('start: ')
                            #print(ref_x,left_shoulder[0],right_shoulder[0],ref_y,left_shoulder[1],right_shoulder[1],ref_d,left_shoulder[2],right_shoulder[2] )
                            #print(ref_x,left_shoulder[0]] )
                        
                        #calibrate all frames, FLIP left hand with right hand, calibrate flipped
                        frame=0
                        for frame in range(starting,ending+1):
                            src_npy_path=f'{src_trial_folder}/{frame}.npy'
                            src_npy=np.load(src_npy_path);
                            srcKeypoints3D=src_npy.reshape(-1,3)

                            if(len(srcKeypoints3D) ==543 ):
                                pass
                            else:
                                print('LandMark error during npy')
                            
                            if (lhFlag == True):
                                flip_src_npy=np.load(src_npy_path);
                                flip_srcKeypoints3D=flip_src_npy.reshape(-1,3)
                                flip_ref_x = (flip_srcKeypoints3D[11][0]+flip_srcKeypoints3D[12][0])/2
                            #print(srcKeypoints3D)
                            #normalize the keypoints
                            #print(srcKeypoints3D[1628])
                            
                            #print(ref_x)
                            
                            
                            #print(ref_y)
                            #calibratedNp=np.array={}
                            #deal with missing values also........
                            keypoint_i=0
                            for keypoint_i in range(0,543):

                                src3d=srcKeypoints3D[keypoint_i]
                                x=src3d[0]
                                y=src3d[1]
                                d=src3d[2]
                                if(x<=0.0 and y <=0.0 and d<=0.0) :
                                   #missing poing
                                   continue

                                src3d[0]=x-ref_x
                                src3d[1]=y-ref_y
                                src3d[2]=d-ref_d

                                if(lhFlag == True):
                                    #flip on original data only hand points last 21 and then 21 points
                                    # Swapping not done yet
                                    mirror_points={13,14,15,16,17,18,19.20,21,22} 
                                    if(keypoint_i in mirror_points or keypoint_i >=501):
                                        flip_src3d=flip_srcKeypoints3D[keypoint_i]
                                        flip_src3d[0]=flip_src3d[0]-flip_ref_x
                                        flip_src3d[0]=-flip_src3d[0]
                                        flip_src3d[0]=flip_ref_x+flip_src3d[0] 
                                                               
                            #for finished        
                                
                                
                                

                            dest_trial_npy_path = dest_trial_folder+'/'+str(frame)+'.npy'
                            # np.save(dest_trial_npy_path,srcKeypoints3D.flatten())
                            np.save(dest_trial_npy_path,makeRelativeAndFlatten(srcKeypoints3D))
                            # exchange hands and shoulder point for LeftHand Orientation
                            if (lhFlag == True):
                                #swap hands and related pose points

                                swap_points={11,13,15,17,19,21} 
                                sp=0
                                for sp in swap_points:
                                    left_x= flip_srcKeypoints3D[sp][0]
                                    left_y= flip_srcKeypoints3D[sp][1]
                                    left_d= flip_srcKeypoints3D[sp][2]
                                
                                    right_x= flip_srcKeypoints3D[sp+1][0]
                                    right_y= flip_srcKeypoints3D[sp+1][1]
                                    right_d= flip_srcKeypoints3D[sp+1][2]

                                                               
                                    flip_srcKeypoints3D[sp][0] =right_x
                                    flip_srcKeypoints3D[sp][1] =right_y
                                    flip_srcKeypoints3D[sp][2] =right_d
                                
                                    flip_srcKeypoints3D[sp+1][0]= left_x
                                    flip_srcKeypoints3D[sp+1][1]= left_y
                                    flip_srcKeypoints3D[sp+1][2]= left_d

                                #left right hand swap
                                hpoint_i=0
                                for hpoint_i in range(0,21):
                                    left_x=flip_srcKeypoints3D[501+hpoint_i][0]
                                    left_y=flip_srcKeypoints3D[501+hpoint_i][1]
                                    left_d=flip_srcKeypoints3D[501+hpoint_i][2]
                                    
                                    right_x=flip_srcKeypoints3D[522+hpoint_i][0]
                                    right_y=flip_srcKeypoints3D[522+hpoint_i][1]
                                    right_d=flip_srcKeypoints3D[522+hpoint_i][2]

                                    flip_srcKeypoints3D[501+hpoint_i][0] =right_x
                                    flip_srcKeypoints3D[501+hpoint_i][1] =right_y
                                    flip_srcKeypoints3D[501+hpoint_i][2] =right_d

                                    flip_srcKeypoints3D[522+hpoint_i][0] =left_x
                                    flip_srcKeypoints3D[522+hpoint_i][1] =left_y
                                    flip_srcKeypoints3D[522+hpoint_i][2] =left_d
                                                                        
                                    
                                    

                                # flip_dest_trial_npy_path = flip_dest_trial_folder+'/'+str(frame)+'.npy'
                                
                                # np.save(flip_dest_trial_npy_path,flip_srcKeypoints3D.flatten())
                                # np.save(flip_dest_trial_npy_path,makeRelativeAndFlatten(flip_srcKeypoints3D))
                                
                                #lefthand flip without calibration finished

                                # do calibration for this frame
                                handkey_i=0
                                for handkey_i in range(0,543):
                                    flip_src3d=flip_srcKeypoints3D[handkey_i]
                                    flip_x=flip_src3d[0]
                                    flip_y=flip_src3d[1]
                                    flip_d=flip_src3d[2]
                                    #missing value
                                    if(flip_x<=0.0 and flip_y <=0.0 and flip_d <=0.0):
                                        continue
                                    flip_src3d[0]=flip_x-ref_x
                                    flip_src3d[1]=flip_y-ref_y
                                    flip_src3d[2]=flip_d-ref_d
                                flip_cal_dest_trial_npy_path = flip_cal_dest_trial_folder+'/'+str(frame)+'.npy'
                                #print(flip_dest_trial_npy_path)
                                np.save(flip_cal_dest_trial_npy_path,makeRelativeAndFlatten(flip_srcKeypoints3D))
                            #if lhFlag True did flip and calibrations
                            #shutil.copy(src_trial_folder+'/'+str(frame)+'.npy', dest_trial_folder+'/'+str(frame)+'.npy')
                        # loop for frames of a trial
                    #loop for trials
                    
                        
                        



In [22]:
def insertionSort(array):

    for step in range(1, len(array)):
        key = array[step]
        j = step - 1
        
        # Compare key with each element on the left of it until an element smaller than it is found
        # For descending order, change key<array[j] to key>array[j].        
        while j >= 0 and key < array[j]:
            array[j + 1] = array[j]
            j = j - 1
        
        # Place key at after the element just smaller than it.
        array[j + 1] = key

In [23]:
def my_flatten(xss):
    return[x for xs in xss for x in xs]

In [24]:
# generating index for class labels
# the index will be used as label in ML

# class one hot encoding
classAnnotation={}
for data in datas:
    for word in data:
        try:
            classAnnotation[word]        
        except:
            classAnnotation[word]={}
        try:
            classAnnotation[word]['ClassLabel']        
        except:
            classAnnotation[word]['ClassLabel'] ={}
        classAnnotation[word]['ClassLabel'] =word

print(len(classAnnotation))
print(classAnnotation)

numbers = []
for cl in classAnnotation:
    classLabel=classAnnotation[cl]['ClassLabel']
    temp = int(classLabel[1:])
    numbers.append(temp)
insertionSort(numbers)
print(numbers)

classIndex=0
for num in numbers:
    classLabel='W'+str(num)
    try:
        classAnnotation[classLabel]['ClassNumer']
    except:
        classAnnotation[classLabel]['ClassNumer']={}
    classAnnotation[classLabel]['ClassNumer'] =classIndex
    classIndex =classIndex+1

NO_CLASSES =len(classAnnotation)
print(classIndex)
print(classAnnotation)


60
{'W355': {'ClassLabel': 'W355'}, 'W356': {'ClassLabel': 'W356'}, 'W215': {'ClassLabel': 'W215'}, 'W216': {'ClassLabel': 'W216'}, 'W19': {'ClassLabel': 'W19'}, 'W20': {'ClassLabel': 'W20'}, 'W213': {'ClassLabel': 'W213'}, 'W214': {'ClassLabel': 'W214'}, 'W91': {'ClassLabel': 'W91'}, 'W92': {'ClassLabel': 'W92'}, 'W357': {'ClassLabel': 'W357'}, 'W358': {'ClassLabel': 'W358'}, 'W3': {'ClassLabel': 'W3'}, 'W4': {'ClassLabel': 'W4'}, 'W351': {'ClassLabel': 'W351'}, 'W352': {'ClassLabel': 'W352'}, 'W11': {'ClassLabel': 'W11'}, 'W12': {'ClassLabel': 'W12'}, 'W359': {'ClassLabel': 'W359'}, 'W360': {'ClassLabel': 'W360'}, 'W37': {'ClassLabel': 'W37'}, 'W38': {'ClassLabel': 'W38'}, 'W99': {'ClassLabel': 'W99'}, 'W100': {'ClassLabel': 'W100'}, 'W49': {'ClassLabel': 'W49'}, 'W50': {'ClassLabel': 'W50'}, 'W43': {'ClassLabel': 'W43'}, 'W44': {'ClassLabel': 'W44'}, 'W47': {'ClassLabel': 'W47'}, 'W48': {'ClassLabel': 'W48'}, 'W1': {'ClassLabel': 'W1'}, 'W2': {'ClassLabel': 'W2'}, 'W93': {'ClassLabe

In [25]:
#generating npy array for ML


LEFT_HAND_ORIENTATION ='LeftHand'

for data in datas:
    for word in data:
        for user in data[word]:
            for orientation in data[word][user]:
                 
                 
                 for fileName in data[word][user][orientation]:
                    
                    frameRate = data[word][user][orientation][fileName]['FrameRate']
                    no_trials = len(data[word][user][orientation][fileName]['trials'])
                    
                    #override the trial count as sometime that is missing in annotation
                    data[word][user][orientation][fileName]['no_of_trials']= no_trials

                    if frameRate == '30' or frameRate == '24' or frameRate == '15':
                        pass
                    else:
                        print (fileName,' frame rate error in NPY FRAME CONVERSION')

                    
                    
                

                    trainorTest = 'Training'
                    if user in testUser:
                        trainorTest='Test'
                    
                    
                    ori_folder = orientation
                    if orientation == LEFT_HAND_ORIENTATION:
                        data_path_org_left = f'{numerical_data_path}/{relativePath}/{trainorTest}/{ori_folder}/{word}/{user}/{fileName}'
                        dest_path_org_left = f'{numerical_data_path}/{relativePath}/ML/{trainorTest}/{orientation}_NO_FLIP'
                        ori_folder ='LeftHand_FLIPPED_CALIB'
                    
                    data_path = f'{numerical_data_path}/{relativePath}/{trainorTest}/{ori_folder}/{word}/{user}/{fileName}'

                    dest_path = f'{numerical_data_path}/{relativePath}/ML/{trainorTest}/{orientation}'
                    #classLabel = word
                    classIndex =classAnnotation[word]['ClassNumer']
                    #print(classLabel,' ',classIndex)

                   
                    
                    #frame_dup_counter= frameDupRate
                    trial=0
                    for trial in range(0,no_trials):
                   
                        trial_path=f'{data_path}/t{trial}'
                        if orientation == 'LeftHand':
                            trial_path_org_left=f'{data_path_org_left}/t{trial}'

                        #print(trial_path,' ',classIndex)
                        starting = data[word][user][orientation][fileName]['trials'][str(trial)]['starting']
                        ending = data[word][user][orientation][fileName]['trials'][str(trial)]['ending']
                        npy_array=[]
                        if orientation == LEFT_HAND_ORIENTATION:
                            npy_array_org_left=[]
                        
                        fcount=0
                        dupcount=0
                        npy_frame=0
                        for npy_frame in range(starting,ending+1):
                            fcount=fcount+1;
                            if (fcount+dupcount) > MAX_FRAME:
                                print('Frame Excceded')
                                continue

                            npy_frame_path =f'{trial_path}/{npy_frame}.npy'
                                                           
                            npy=np.load(npy_frame_path)
                            #print(len(npy))
                            if (len(npy) >1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK')
                            npy_array.append(npy)
                            

                            if orientation == LEFT_HAND_ORIENTATION:
                                npy_frame_path_org_left =f'{trial_path_org_left}/{npy_frame}.npy'
                                npy_org_left=np.load(npy_frame_path_org_left)
                                npy_array_org_left.append(npy_org_left)
                                

                            #npy_array_oneIns[fcount]=npy
                            #npy.reshape(-1,3)
                            
                            

                            
                            
                            if frameRate =='15':
                                #npy dup
                                npy_array.append(npy)
                                
                                if orientation == LEFT_HAND_ORIENTATION:
                                    npy_array_org_left.append(npy_org_left)
                                    
                                dupcount =dupcount+1
                                
                            elif frameRate == '24' and fcount % 4 == 0:
                                #npy dup
                                npy_array.append(npy)
                                
                                if orientation == LEFT_HAND_ORIENTATION:
                                    npy_array_org_left.append(npy_org_left)
                                    
                                dupcount =dupcount+1
                        #for srtaring ending frame of a trial
                        #check for zero padding needed
                        #print(np.size(npy))
                        #zf=MAX_FRMAE-fcount-dupcount
                        #print (MAX_FRMAE, zf, fcount,dupcount)
                        
                        fcount =fcount+dupcount
                        #print('count: ',count)
                        count=0
                        for count in range(fcount,MAX_FRAME):
                            npy =np.zeros(543*3)
                            npy_array.append(npy)
                            
                            if orientation == LEFT_HAND_ORIENTATION:
                                npy =np.zeros(543*3)
                                npy_array_org_left.append(npy)
                        #print('end count: ',count)    
                        #now class index
                        #class_array.append(classIndex);
                        #print(len(npy_array))
                        dest_class_path=f'{dest_path}/{str(classIndex)}'
                        if os.path.exists(dest_class_path):
                            pass
                        else:
                            os.makedirs(dest_class_path)
                        
                        dest_file_path =f'{dest_class_path}/{classIndex}_{user}_{trial}_{fileName}.npy'                                            
                        np.save(dest_file_path,my_flatten(npy_array))

                        if orientation == LEFT_HAND_ORIENTATION: 
                            dest_class_path_org_left=f'{dest_path_org_left}/{str(classIndex)}'
                            if os.path.exists(dest_class_path_org_left):
                                pass
                            else:
                                os.makedirs(dest_class_path_org_left)
                            
                            dest_file_path_org_left =f'{dest_class_path_org_left}/{classIndex}_{user}_{trial}_{fileName}.npy'                        
                            np.save(dest_file_path_org_left,my_flatten(npy_array_org_left))                   
                     
                                       

                            
                            
                            




In [26]:
#generating npy array for ML 
#PROLONG THE SHORT GESTURE BY DUPLICATING


LEFT_HAND_ORIENTATION ='LeftHand'

for data in datas:
    for word in data:
        for user in data[word]:
            for orientation in data[word][user]:
                 
                 
                 for fileName in data[word][user][orientation]:
                    
                    frameRate = data[word][user][orientation][fileName]['FrameRate']
                    no_trials = len(data[word][user][orientation][fileName]['trials'])
                    
                    #override the trial count as sometime that is missing in annotation
                    data[word][user][orientation][fileName]['no_of_trials']= no_trials

                    if frameRate == '30' or frameRate == '24' or frameRate == '15':
                        pass
                    else:
                        print (fileName,' frame rate error in NPY FRAME CONVERSION')

                    if (len(npy) >1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK')
                    if (len(npy) <1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK',fileName)
                   

                    trainorTest = 'Training'
                    if user in testUser:
                        trainorTest='Test'
                    
                    
                    ori_folder = orientation
                    if orientation == LEFT_HAND_ORIENTATION:
                        data_path_org_left = f'{numerical_data_path}/{relativePath}/{trainorTest}/{ori_folder}/{word}/{user}/{fileName}'
                        dest_path_org_left = f'{numerical_data_path}/{relativePath}/ML_PROLONG/{trainorTest}/{orientation}_NO_FLIP'
                        ori_folder ='LeftHand_FLIPPED_CALIB'
                    
                    data_path = f'{numerical_data_path}/{relativePath}/{trainorTest}/{ori_folder}/{word}/{user}/{fileName}'

                    dest_path = f'{numerical_data_path}/{relativePath}/ML_PROLONG/{trainorTest}/{orientation}'
                    #classLabel = word
                    classIndex =classAnnotation[word]['ClassNumer']
                    #print(classLabel,' ',classIndex)

                   
                    
                    #frame_dup_counter= frameDupRate
                    trial=0
                    for trial in range(0,no_trials):
                   
                        trial_path=f'{data_path}/t{trial}'
                        if orientation == 'LeftHand':
                            trial_path_org_left=f'{data_path_org_left}/t{trial}'

                        #print(trial_path,' ',classIndex)
                        starting = data[word][user][orientation][fileName]['trials'][str(trial)]['starting']
                        ending = data[word][user][orientation][fileName]['trials'][str(trial)]['ending']
                        npy_array=[]
                        if orientation == LEFT_HAND_ORIENTATION:
                            npy_array_org_left=[]
                        
                        
                        tf =math.ceil((ending-starting+1)*30/int(frameRate))
                        dupRatio =MAX_FRAME//tf
                        
                        fcount=0
                        dupcount=0
                        spreadCount=0
                        npy_frame=0
                        for npy_frame in range(starting,ending+1):
                            fcount=fcount+1;
                            if (fcount+dupcount) > MAX_FRAME:
                                print('Frame Excceded')
                                continue

                            npy_frame_path =f'{trial_path}/{npy_frame}.npy'
                                                           
                            npy=np.load(npy_frame_path)
                            #print(len(npy))
                            if (len(npy) >1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK')
                            if (len(npy) <1629):
                                print('NPY WRONG POSE/VISIBILITY DATA INCLUDED FOR ZERO POSE LANDMARK',fileName)
                            
                            
                            npy_array.append(npy)
                            #print('DUP ratio: ',dupRatio)
                            dup=1;
                            for dup in range(1,dupRatio):
                                # print ('duplicate times: ',dup, trial,word,user,dupRatio)
                                 npy_array.append(npy)
                                 dupcount=dupcount+1
                            
                            #uniformly spread/duplicate the frames upto max frame
                            scFlag=False
                            if(spreadCount<(MAX_FRAME -tf*dupRatio)):
                                 #print('spread: ',trial,word,user)
                                 npy_array.append(npy)
                                 spreadCount =spreadCount+1
                                 scFlag=True
                                 dupcount=dupcount+1
                                
                            

                            if orientation == LEFT_HAND_ORIENTATION:
                                npy_frame_path_org_left =f'{trial_path_org_left}/{npy_frame}.npy'
                                npy_org_left=np.load(npy_frame_path_org_left)
                                npy_array_org_left.append(npy_org_left)
                                
                                dup=1
                                for dup in range(1,dupRatio):
                                    npy_array_org_left.append(npy_org_left)
                                #uniformly spread/duplicate the frames upto max frame
                                
                                if scFlag==True:
                                    npy_array_org_left.append(npy_org_left)
                                 
                                

                            #npy_array_oneIns[fcount]=npy
                            #npy.reshape(-1,3)
                            
                            

                            
                            
                            if frameRate =='15':
                                #npy dup
                                npy_array.append(npy)
                                dupcount =dupcount+1
                                
                                dup=1
                                for dup in range(1,dupRatio):
                                    npy_array.append(npy)
                                    dupcount=dupcount+1
                                
                                if orientation == LEFT_HAND_ORIENTATION:
                                    npy_array_org_left.append(npy_org_left)
                                    dup =1
                                    for dup in range(1,dupRatio):
                                        npy_array_org_left.append(npy_org_left)
                                
                                
                            elif frameRate == '24' and fcount % 4 == 0:
                                #npy dup
                                npy_array.append(npy)
                                dupcount =dupcount+1
                                dup=1
                                for dup in range(1,dupRatio):
                                    npy_array.append(npy)
                                    dupcount=dupcount+1

                                if orientation == LEFT_HAND_ORIENTATION:
                                    npy_array_org_left.append(npy_org_left)
                                    dup=1
                                    for dup in range(1,dupRatio):
                                        npy_array_org_left.append(npy_org_left)
                                
                        #for srtaring ending frame of a trial
                        #check for zero padding needed
                        #print(np.size(npy))
                        #zf=MAX_FRMAE-fcount-dupcount
                        #print (MAX_FRMAE, zf, fcount,dupcount)
                        
                        fcount =fcount+dupcount
                        #print('count: ',count)
                        count=0
                        for count in range(fcount,MAX_FRAME):
                            npy =np.zeros(543*3)
                            npy_array.append(npy)
                            
                        
                        if orientation == LEFT_HAND_ORIENTATION:
                            count=0
                            for count in range(fcount,MAX_FRAME):
                                npy2 =np.zeros(543*3)
                                npy_array_org_left.append(npy2)
                        #print('end count: ',count)    
                        #now class index
                        #class_array.append(classIndex);
                        #print(len(npy_array))
                        dest_class_path=f'{dest_path}/{str(classIndex)}'
                        if os.path.exists(dest_class_path):
                            pass
                        else:
                            os.makedirs(dest_class_path)
                        
                        dest_file_path =f'{dest_class_path}/{classIndex}_{user}_{trial}_{fileName}.npy'                                            
                        np.save(dest_file_path,my_flatten(npy_array))

                        if orientation == LEFT_HAND_ORIENTATION: 
                            #print(len(npy_array_org_left))
                            dest_class_path_org_left=f'{dest_path_org_left}/{str(classIndex)}'
                            if os.path.exists(dest_class_path_org_left):
                                pass
                            else:
                                os.makedirs(dest_class_path_org_left)
                            
                            dest_file_path_org_left =f'{dest_class_path_org_left}/{classIndex}_{user}_{trial}_{fileName}.npy'                        
                            np.save(dest_file_path_org_left,my_flatten(npy_array_org_left))            

                        
                      
                    

                         
                            
                         




In [27]:
import os
import numpy as np


def normalizeData (norm_path, crossValidationDataPaths,testPaths):
    print("hello")
    PER_FRAME_FEATURE =1629
    min=np.zeros (PER_FRAME_FEATURE)
    max=np.zeros (PER_FRAME_FEATURE)
    

    for i in range (0,PER_FRAME_FEATURE):
        min[i]= 9
        max[i]= -9

    ml_instances_paths=[]
    for path in crossValidationDataPaths:   
        print('training path: ',path) 
        classFolders= os.listdir(path)
        for classFolder in classFolders:
            classFolder_path=f'{path}/{classFolder}'
            classTrials =os.listdir(classFolder_path)
            for trial in classTrials:
                trialPath =f'{classFolder_path}/{trial}'
                ml_instances_paths.append(trialPath)           
                npy =np.load(trialPath)            
                npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)
                feature =0
                for feature in range(0,PER_FRAME_FEATURE):
                    temp_min = np.min(npy_matrix[:,feature])
                    temp_max = np.max(npy_matrix[:,feature])
                
                    if temp_max>max[feature]:
                        max[feature] =temp_max
                    if temp_min < min[feature]:
                        min[feature] =temp_min
    
    test_paths=[]
    for path in testPaths:    
        print('test path: ',path) 
        classFolders= os.listdir(path)
        for classFolder in classFolders:
            classFolder_path=f'{path}/{classFolder}'
            classTrials =os.listdir(classFolder_path)
            for trial in classTrials:
                trialPath =f'{classFolder_path}/{trial}'            
                test_paths.append(trialPath)
                #generate min-max
                npy =np.load(trialPath)
                npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)          
            
                feature=0
                for feature in range(0,PER_FRAME_FEATURE):
                    temp_min = np.min(npy_matrix[:,feature])
                    temp_max = np.max(npy_matrix[:,feature])
                
                    if temp_max>max[feature]:
                        max[feature] =temp_max
                    if temp_min < min[feature]:
                        min[feature] =temp_min

    print('Normalization starting')

    for ml_instances_path in ml_instances_paths:         
        npy =np.load(ml_instances_path)
        npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)
        feature=0
        for feature in range(0,PER_FRAME_FEATURE):          
            my_range = max[feature]-min[feature]        
            npy_matrix[:,feature] = (npy_matrix[:,feature]*100000 -min[feature]*100000)/(my_range*100000+0.00001)     
        
        #save in file
        # toDest_path=f'{norm_path}/{ml_instances_path}'
    
        #create the target folder
        label = ml_instances_path.split('/')
        dir_path =''
        i=0
        for i in range(1,len(label)-1): #omit ../
            dir_path=dir_path+label[i]+'/'

        dir_path=f'{norm_path}/{dir_path}'
        if os.path.exists(dir_path):
            pass           
                           
        else: 
            os.makedirs(dir_path)    

        toDest_path =f'{dir_path}/{label[len(label)-1]}'
        np.save(toDest_path,npy_matrix.flatten())

    for ml_instances_path in test_paths:  
        npy =np.load(ml_instances_path)
        npy_matrix=npy.reshape(-1,PER_FRAME_FEATURE)
        feature =0
        for feature in range(0,PER_FRAME_FEATURE):
            my_range =max[feature]-min[feature]
            npy_matrix[:,feature] = (npy_matrix[:,feature]*100000 -min[feature]*100000)/(my_range*100000+0.00001)  

        #save in file
        # toDest_path=f'{norm_path}/{ml_instances_path}'

        label = ml_instances_path.split('/')
        dir_path =''
        i=0
        for i in range(1,len(label)-1):  #omit ../
            dir_path=dir_path+label[i]+'/'

        dir_path=f'{norm_path}/{dir_path}'
        if os.path.exists(dir_path):
            pass                                                   
        else: 
            os.makedirs(dir_path)
        
        toDest_path =f'{dir_path}/{label[len(label)-1]}'
        np.save(toDest_path,npy_matrix.flatten())

    




In [ ]:
norm_path ='../RELATIVE_NORMALIZED'

#USE ONE OF EACH TWO LINES
crossValidationDataPaths=['../NumericalData/RELATIVE/ML/Training/RightHand','../NumericalData/RELATIVE/ML/Training/LeftHand']
testPaths=['../NumericalData/RELATIVE/ML/Test/RightHand','../NumericalData/RELATIVE/ML/Test/LeftHand']
normalizeData(norm_path,crossValidationDataPaths,testPaths)

crossValidationDataPaths=['../NumericalData/RELATIVE/ML/Training/LeftHand_NO_FLIP']
testPaths=['../NumericalData/RELATIVE/ML/Test/LeftHand_NO_FLIP']
normalizeData(norm_path,crossValidationDataPaths,testPaths)

crossValidationDataPaths=['../NumericalData/RELATIVE/ML_PROLONG/Training/RightHand','../NumericalData/RELATIVE/ML_PROLONG/Training/LeftHand']
testPaths=['../NumericalData/RELATIVE/ML_PROLONG/Test/RightHand','../NumericalData/RELATIVE/ML_PROLONG/Test/LeftHand']
normalizeData(norm_path,crossValidationDataPaths,testPaths)

crossValidationDataPaths=['../NumericalData/RELATIVE/ML_PROLONG/Training/LeftHand_NO_FLIP']
testPaths=['../NumericalData/RELATIVE/ML_PROLONG/Test/LeftHand_NO_FLIP']
normalizeData(norm_path,crossValidationDataPaths,testPaths)